In [1]:
import sys
sys.path.insert(0, '/home/knoop/local/hilde')

In [2]:
import numpy as np

from matplotlib import pyplot as plt

from hilde.trajectory import reader
from hilde.konstanten import omega_to_THz

from hilde.harmonic_analysis import parse_tdep_forceconstant, HarmonicAnalysis

from hilde.harmonic_analysis.displacements import get_U

from hilde.harmonic_analysis.normal_modes import u_I_to_u_s, map_L_to_i, projector as mode_projector

from hilde.helpers import progressbar

In [3]:
traj = reader('no_drift.yaml')

.. no_drift.yaml parsed in 3.433s


In [4]:
fcs, lps = parse_tdep_forceconstant('outfile.forceconstant')

ha = HarmonicAnalysis(traj.primitive, traj.supercell, fcs, lps)

Parse force constants from
  outfile.forceconstant
.. remap representation for supercell:  False
.. Number of atoms:   2
.. Real space cutoff: 3.660 \AA
.. Number of lattice points: 54 (33 unique)
.. time elapsed: 0.002s
Set up harmonic analysis for MgO:
.. found 32 (31) lattice points in 0.011s
.. matched 64 positions in supercell and primitive cell in 0.088s
.. found 32 (31) lattice points in 0.002s
.. time elapsed: 0.277s


In [5]:
N = len(traj)
atoms0 = traj.supercell
kwargs = {'q_points': ha.q_points, 'lattice_points': ha.lattice_points_supercell, 'eigenvectors': ha.eigenvectors,
        'indeces': ha.I_to_iL}

In [6]:
u_I = get_U(traj[1], atoms0)

iL = ha.I_to_iL
RLs = ha.lattice_points_supercell[ha.I_to_iL[:, 1]]
RLs.shape

(64, 3)

In [7]:
na = len(ha.supercell)
nq, ns = ha.eigenvectors.shape[:2]
ievs = ha.eigenvectors.conj().swapaxes(1, 2).reshape(nq, ns, ns // 3, 3)

ievs.shape, ievs[:, :, ha.I_to_iL[:, 0], :].shape, (ha.q_points @ RLs.T).repeat(3, axis=1).shape

((32, 6, 2, 3), (32, 6, 64, 3), (32, 192))

In [8]:
aug_ev = 1/(nq**.5) * np.exp(-2j * np.pi * (ha.q_points @ RLs.T).repeat(3, axis=1).reshape(nq, na, -1))[:, None, :, :] *  ievs[:, :, ha.I_to_iL[:, 0], :]

In [9]:
ev = mode_projector(**kwargs, flat=False)
ev_flat = mode_projector(**kwargs, flat=True)
ev.shape, aug_ev.shape, ev_flat.shape

((32, 6, 64, 3), (32, 6, 64, 3), (32, 6, 192))

In [10]:
np.linalg.norm(ev - aug_ev)

0.0

In [11]:
%%timeit
us =  (aug_ev * u_I).sum(axis=(2, 3))

220 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%%timeit
us_flat = ev_flat @ u_I.flatten()

32.7 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
%%timeit
usp = u_I_to_u_s(u_I, **kwargs)

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

...

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector i

In [14]:
# are they equal?
us =  (aug_ev * u_I).sum(axis=(2, 3))
us_flat = (ev_flat @ u_I.flatten())
usp = u_I_to_u_s(u_I, **kwargs)
np.linalg.norm(usp - us), np.linalg.norm(usp - us_flat)

** Warning from file hilde/harmonic_analysis/normal_modes.py, line 183, function u_I_to_u_s:
--> DEPRECATED: Usenormal_modes.projector instead



(1.7434926320724184e-15, 1.7434926320724184e-15)